In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Data
df = pd.read_csv('resources/DATA.csv')

# Target Columns
target_columns = [
    'HN',
    'HNE',
    'HE',
    'HSE',
    'HS',
    'HSW',
    'HW',
    'HNW',
    'MN',
    'MNE',
    'ME',
    'MSE',
    'MS',
    'MSW',
    'MW',
    'MNW',
    'LN',
    'LNE',
    'LE',
    'LSE',
    'LS',
    'LSW',
    'LW',
    'LNW'
]

# Convert colors to numbers
map = {
    'gray': 0,
    'green': 1,
    'yellow': 2,
    'orange': 3,
    'red': 4,
    'black': 5,    
}

# For target columns
for col in target_columns:
    df[col] = df[col].map(map)

# Make a new column be the max risk of the target columns
df['max_risk'] = df[target_columns].max(axis=1)
target_columns += ['max_risk']
    
input_columns = [
    'ALTA_Precip_(tenths_mm)',
    'ALTA_Snowfall_(mm)',
    'ALTA_Snow_Depth_(mm)',
    'ALTA_Max_Temp_(tenths_C)',
    'ALTA_Min_Temp_(tenths_C)',
    'ALTA_Multiday_Prec_Days',
    'ALTA_Multiday_Snowfall_Days',
    'ALTA_Multiday_Prec_Days_(tenths_mm)',
    'ALTA_Multiday_Snowfall',
    'ALTA_Temp_at_Observation_(tenths_C)',
    'ALTA_Fog_Ice_Fog',
    'ALTA_Heavy_Fog_Freezing_Fog',
    'ALTA_Ice_Pellets_Sleet',
    'ALTA_Hail',
    'ALTA_Glaze_Rime',
    'ALTA_Weather_Type_07',
    'ALTA_Smoke_Haze',
    'ALTA_Blowing_Drifting_Snow',
    'ALTA_High_Winds',
    'ALTA_Snow_Pellets_Ice_Crystals',
    'BRIGHTON_Precip_(tenths_mm)',
    'BRIGHTON_Snow_Depth_(mm)',
    'BRIGHTON_Max_Temp_(tenths_C)',
    'BRIGHTON_Min_Temp_(tenths_C)',
    'BRIGHTON_Avg_Daily_Temp_(tenths_C)',
    'BRIGHTON_Temp_at_Observation_(tenths_C)',
    'BRIGHTON_Snow_on_Ground_(tenths_mm)',
    'THAYNES_Precip_(tenths_mm)',
    'THAYNES_Snow_Depth_(mm)',
    'THAYNES_Max_Temp_(tenths_C)',
    'THAYNES_Min_Temp_(tenths_C)',
    'THAYNES_Avg_Daily_Temp_(tenths_C)',
    'THAYNES_Temp_at_Observation_(tenths_C)',
    'THAYNES_Snow_on_Ground_(tenths_mm)',
    'PARLEY_SUMMIT_Precip_(tenths_mm)',
    'PARLEY_SUMMIT_Snow_Depth_(mm)',
    'PARLEY_SUMMIT_Max_Temp_(tenths_C)',
    'PARLEY_SUMMIT_Min_Temp_(tenths_C)',
    'PARLEY_SUMMIT_Temp_at_Observation_(tenths_C)',
    'PARLEY_SUMMIT_Snow_on_Ground_(tenths_mm)'
]

# add data from previous day
for col in target_columns:
    input_columns.append(col + '_prev')
    df[col + '_prev'] = df[col].shift(1)

# Fill NA with 0
df = df.fillna(0)

X = df[input_columns]
y = df['max_risk']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Classifiers

In [77]:
results = {}

In [78]:
# Just for my own curiosity I'll run a few quick models
# MLP, Decision Tree, KNN
# MLP Classifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

mlp_accuracies = {}
params250 = [(250, 250, 250), (250, 250, 250, 250), (250, 250, 250, 250, 250), (250, 250, 250, 250, 250, 250), (250, 250, 250, 250, 250, 250, 250)]
params100 = [(100, 100, 100), (100, 100, 100, 100), (100, 100, 100, 100, 100), (100, 100, 100, 100, 100, 100), (100, 100, 100, 100, 100, 100, 100)]
params32 = [(32, 32, 32), (32, 32, 32, 32), (32, 32, 32, 32, 32), (32, 32, 32, 32, 32, 32)]
params10 = [(10, 10, 10), (10, 10, 10, 10), (10, 10, 10, 10, 10), (10, 10, 10, 10, 10, 10)]
params = params100 + params32 + params10
for param in params:
    mlp = MLPClassifier(hidden_layer_sizes=param, early_stopping=True, max_iter=1000)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    mlp_accuracies[param] = accuracy_score(y_test, y_pred)
    results[('MLPClassifer', param)] = {
        'model': mlp,
        'accuracy': accuracy_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print baseline accuracy
print("Baseline: ", y_test.value_counts().max() / len(y_test)) 

# Print MLP accuracies
print("Max: ", max(mlp_accuracies, key=mlp_accuracies.get), mlp_accuracies[max(mlp_accuracies, key=mlp_accuracies.get)])
# for param in params:
#     print(param, mlp_accuracies[param])

Baseline:  0.42394822006472493
Max:  (100, 100, 100, 100, 100, 100, 100) 0.5307443365695793


In [79]:

# Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

dt_accuracies = {}
params = range(1, 20)
for param in params:
    dt = DecisionTreeClassifier(max_depth=param)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    dt_accuracies[param] = accuracy_score(y_test, y_pred)
    results[('DecisionTreeClassifier', param)] = {
        'model': dt,
        'accuracy': accuracy_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred)
    }

# Print Decision Tree accuracies
print("Max: ", max(dt_accuracies, key=dt_accuracies.get), dt_accuracies[max(dt_accuracies, key=dt_accuracies.get)])
# for param in params:
#     print(param, dt_accuracies[param])

Max:  3 0.7022653721682848


In [80]:

# KNN Classifier
from sklearn.neighbors import KNeighborsClassifier

knn_accuracies = {}
params = range(1, 20)
for param in params:
    knn = KNeighborsClassifier(n_neighbors=param)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    knn_accuracies[param] = accuracy_score(y_test, y_pred)
    results[('KNeighborsClassifier', param)] = {
        'model': knn,
        'accuracy': accuracy_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print KNN accuracies
print("Max: ", max(knn_accuracies, key=knn_accuracies.get), knn_accuracies[max(knn_accuracies, key=knn_accuracies.get)])
# for param in params:
#     print(param, knn_accuracies[param])

Max:  1 0.6375404530744336


In [81]:

# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

rf_accuracies = {}
params = range(1, 20)
for param in params:
    rf = RandomForestClassifier(max_depth=param)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    rf_accuracies[param] = accuracy_score(y_test, y_pred)
    results[('RandomForestClassifier', param)] = {
        'model': rf,
        'accuracy': accuracy_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print Random Forest accuracies
print("Max: ", max(rf_accuracies, key=rf_accuracies.get), rf_accuracies[max(rf_accuracies, key=rf_accuracies.get)])
# for param in params:
#     print(param, rf_accuracies[param])

Max:  11 0.7119741100323624


In [82]:

# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gb_accuracies = {}
params = range(5, 6)
for param in params:
    gb = GradientBoostingClassifier(max_depth=param)
    gb.fit(X_train, y_train)
    y_pred = gb.predict(X_test)
    gb_accuracies[param] = accuracy_score(y_test, y_pred)
    results[('GradientBoostingClassifier', param)] = {
        'model': gb,
        'accuracy': accuracy_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print Gradient Boosting accuracies
print("Max: ", max(gb_accuracies, key=gb_accuracies.get), gb_accuracies[max(gb_accuracies, key=gb_accuracies.get)])
# for param in params:
#     print(param, gb_accuracies[param])

Max:  5 0.7119741100323624


In [83]:

# AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier

ab_accuracies = {}
params = range(1, 20)
for param in params:
    ab = AdaBoostClassifier(n_estimators=param)
    ab.fit(X_train, y_train)
    y_pred = ab.predict(X_test)
    ab_accuracies[param] = accuracy_score(y_test, y_pred)
    results[('AdaBoostClassifier', param)] = {
        'model': ab,
        'accuracy': accuracy_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print AdaBoost accuracies
print("Max: ", max(ab_accuracies, key=ab_accuracies.get), ab_accuracies[max(ab_accuracies, key=ab_accuracies.get)])
# for param in params:
#     print(param, ab_accuracies[param])

Max:  11 0.6796116504854369


In [84]:

# XGBoost Classifier
from xgboost import XGBClassifier

xgb_accuracies = {}
params = range(1, 20)
for param in params:
    xgb = XGBClassifier(max_depth=param)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    xgb_accuracies[param] = accuracy_score(y_test, y_pred)
    results[('XGBClassifier', param)] = {
        'model': xgb,
        'accuracy': accuracy_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print XGBoost accuracies
print("Max: ", max(xgb_accuracies, key=xgb_accuracies.get), xgb_accuracies[max(xgb_accuracies, key=xgb_accuracies.get)])
# for param in params:
#     print(param, xgb_accuracies[param])

Max:  5 0.7313915857605178


In [85]:

# SVM Classifier
from sklearn.svm import SVC

svm_accuracies = {}
params = range(1, 20)
for param in params:
    svm = SVC(C=param)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    svm_accuracies[param] = accuracy_score(y_test, y_pred)
    results[('SVC', param)] = {
        'model': svm,
        'accuracy': accuracy_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print SVM accuracies
print("Max: ", max(svm_accuracies, key=svm_accuracies.get), svm_accuracies[max(svm_accuracies, key=svm_accuracies.get)])
# for param in params:
#     print(param, svm_accuracies[param])

Max:  16 0.6051779935275081


In [86]:

# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression

lr_accuracies = {}
#‘lbfgs’, ‘liblinear’, ‘newton-cg’, ‘newton-cholesky’, ‘sag’, ‘saga’
params = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

# preprocess X
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
pX_train = scaler.fit_transform(X_train)
pX_test = scaler.transform(X_test)

for param in params:
    lr = LogisticRegression(solver=param, max_iter=10000)
    lr.fit(pX_train, y_train)
    y_pred = lr.predict(pX_test)
    lr_accuracies[param] = accuracy_score(y_test, y_pred)
    results[('LogisticRegression', param)] = {
        'model': lr,
        'accuracy': accuracy_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print Logistic Regression accuracies
print("Max: ", max(lr_accuracies, key=lr_accuracies.get), lr_accuracies[max(lr_accuracies, key=lr_accuracies.get)])
# for param in params:
#     print(param, lr_accuracies[param])

Max:  sag 0.6731391585760518


In [87]:

# Gaussian Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
gnb_accuracy = accuracy_score(y_test, y_pred)
results[('GaussianNB', '')] = {
    'model': gnb,
    'accuracy': accuracy_score(y_test, y_pred),
    'mse': mean_squared_error(y_test, y_pred)
}
    
# Print Gaussian Naive Bayes accuracies
print(gnb_accuracy)
# for param in params:
#     print(param, gnb_accuracies[param])


0.5598705501618123


# Regressors

In [88]:

# Regressors
# MLP Regressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

mlpr_mse = {}
mlpr_accuracy = {}
params250 = [(250, 250, 250), (250, 250, 250, 250), (250, 250, 250, 250, 250), (250, 250, 250, 250, 250, 250), (250, 250, 250, 250, 250, 250, 250)]
params100 = [(100, 100, 100), (100, 100, 100, 100), (100, 100, 100, 100, 100), (100, 100, 100, 100, 100, 100, 100)]
params32 = [(32, 32, 32), (32, 32, 32, 32), (32, 32, 32, 32, 32), (32, 32, 32, 32, 32, 32)]
params10 = [(10, 10, 10), (10, 10, 10, 10), (10, 10, 10, 10, 10), (10, 10, 10, 10, 10, 10)]
params = params100 + params32 + params10
for param in params:
    mlpr = MLPRegressor(hidden_layer_sizes=param, early_stopping=True, max_iter=1000)
    mlpr.fit(X_train, y_train)
    y_pred = mlpr.predict(X_test)
    mlpr_mse[param] = mean_squared_error(y_test, y_pred)
    mlpr_accuracy[param] = accuracy_score(y_test, y_pred.round())
    results[('MLPRegressor', param)] = {
        'model': mlpr,
        'accuracy': accuracy_score(y_test, y_pred.round()),
        'mse': mean_squared_error(y_test, y_pred)
    }

# Print MLP Regressor accuracies
print("Min: ", min(mlpr_mse, key=mlpr_mse.get), mlpr_mse[min(mlpr_mse, key=mlpr_mse.get)])
print("Max: ", max(mlpr_accuracy, key=mlpr_accuracy.get), mlpr_accuracy[max(mlpr_accuracy, key=mlpr_accuracy.get)])

Min:  (100, 100, 100, 100, 100, 100, 100) 0.5247558660274091
Max:  (100, 100, 100, 100, 100, 100, 100) 0.5372168284789643


In [89]:

# Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor

dtr_mse = {}
dtr_accuracy = {}
params = range(1, 20)
for param in params:
    dtr = DecisionTreeRegressor(max_depth=param)
    dtr.fit(X_train, y_train)
    y_pred = dtr.predict(X_test)
    dtr_mse[param] = mean_squared_error(y_test, y_pred)
    dtr_accuracy[param] = accuracy_score(y_test, y_pred.round())
    results[('DecisionTreeRegressor', param)] = {
        'model': dtr,
        'accuracy': accuracy_score(y_test, y_pred.round()),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print Decision Tree Regressor accuracies
print("Min: ", min(dtr_mse, key=dtr_mse.get), dtr_mse[min(dtr_mse, key=dtr_mse.get)])
print("Max: ", max(dtr_accuracy, key=dtr_accuracy.get), dtr_accuracy[max(dtr_accuracy, key=dtr_accuracy.get)])

Min:  5 0.32694036677592286
Max:  6 0.6990291262135923


In [90]:

# KNN Regressor
from sklearn.neighbors import KNeighborsRegressor

knr_mse = {}
knr_accuracy = {}
params = range(1, 20)
params2 = ['euclidean', 'manhattan', 'chebyshev']

for param in params:
    for param2 in params2:
        knr = KNeighborsRegressor(n_neighbors=param, metric=param2)
        knr.fit(X_train, y_train)
        y_pred = knr.predict(X_test)
        knr_mse[(param, param2)] = mean_squared_error(y_test, y_pred)
        knr_accuracy[(param, param2)] = accuracy_score(y_test, y_pred.round())
        results[('KNeighborsRegressor', (param, param2))] = {
            'model': knr,
            'accuracy': accuracy_score(y_test, y_pred.round()),
            'mse': mean_squared_error(y_test, y_pred)
        }
        
# Print KNN Regressor accuracies
print("Min: ", min(knr_mse, key=knr_mse.get), knr_mse[min(knr_mse, key=knr_mse.get)])
print("Max: ", max(knr_accuracy, key=knr_accuracy.get), knr_accuracy[max(knr_accuracy, key=knr_accuracy.get)])

Min:  (2, 'chebyshev') 0.4199029126213592
Max:  (1, 'manhattan') 0.6504854368932039


In [91]:

# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

rfr_mse = {}
rfr_accuracy = {}
params = range(1, 20)
for param in params:
    rfr = RandomForestRegressor(max_depth=param)
    rfr.fit(X_train, y_train)
    y_pred = rfr.predict(X_test)
    rfr_mse[param] = mean_squared_error(y_test, y_pred)
    rfr_accuracy[param] = accuracy_score(y_test, y_pred.round())
    results[('RandomForestRegressor', param)] = {
        'model': rfr,
        'accuracy': accuracy_score(y_test, y_pred.round()),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print Random Forest Regressor accuracies
print("Min: ", min(rfr_mse, key=rfr_mse.get), rfr_mse[min(rfr_mse, key=rfr_mse.get)])
print("Max: ", max(rfr_accuracy, key=rfr_accuracy.get), rfr_accuracy[max(rfr_accuracy, key=rfr_accuracy.get)])

Min:  16 0.26609003854388097
Max:  12 0.7184466019417476


In [92]:

# XGBoost Regressor
from xgboost import XGBRegressor

xgbr_mse = {}
xgbr_accuracy = {}
params = range(1, 20)
for param in params:
    xgbr = XGBRegressor(max_depth=param)
    xgbr.fit(X_train, y_train)
    y_pred = xgbr.predict(X_test)
    xgbr_mse[param] = mean_squared_error(y_test, y_pred)
    xgbr_accuracy[param] = accuracy_score(y_test, y_pred.round())
    results[('XGBRegressor', param)] = {
        'model': xgbr,
        'accuracy': accuracy_score(y_test, y_pred.round()),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print XGBoost Regressor accuracies
print("Min: ", min(xgbr_mse, key=xgbr_mse.get), xgbr_mse[min(xgbr_mse, key=xgbr_mse.get)])
print("Max: ", max(xgbr_accuracy, key=xgbr_accuracy.get), xgbr_accuracy[max(xgbr_accuracy, key=xgbr_accuracy.get)])

Min:  2 0.2643110258098708
Max:  3 0.6990291262135923


In [93]:

# SVM Regressor
from sklearn.svm import SVR

svr_mse = {}
svr_accuracy = {}
params = range(1, 20)

for param in params:
    svr = SVR(C=param)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    svr_mse[param] = mean_squared_error(y_test, y_pred)
    svr_accuracy[param] = accuracy_score(y_test, y_pred.round())
    results[('SVR', param)] = {
        'model': svr,
        'accuracy': accuracy_score(y_test, y_pred.round()),
        'mse': mean_squared_error(y_test, y_pred)
    }
    
# Print SVM Regressor accuracies
print("Min: ", min(svr_mse, key=svr_mse.get), svr_mse[min(svr_mse, key=svr_mse.get)])
print("Max: ", max(svr_accuracy, key=svr_accuracy.get), svr_accuracy[max(svr_accuracy, key=svr_accuracy.get)])

Min:  19 0.4861573664278051
Max:  18 0.6019417475728155


In [75]:
# DO ALL ABOVE, BUT ALGORITHMICALLY
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

models = [
    MLPClassifier,
    DecisionTreeClassifier,
    KNeighborsClassifier,
    RandomForestClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    XGBClassifier,
    SVC,
    LogisticRegression,
    LinearRegression,
    GaussianNB,
    MLPRegressor,
    DecisionTreeRegressor,
    KNeighborsRegressor,
    RandomForestRegressor,
    XGBRegressor,
    SVR
]

model_accuracies = {}
model_mses = {}

# give parameter options for each model
params = {
    MLPClassifier: [(250, 250, 250), (250, 250, 250, 250), (250, 250, 250, 250, 250), (250, 250, 250, 250, 250, 250), (250, 250, 250, 250, 250, 250, 250),
                    (100, 100, 100), (100, 100, 100, 100), (100, 100, 100, 100, 100), (100, 100, 100, 100, 100, 100, 100),
                    (32, 32, 32), (32, 32, 32, 32), (32, 32, 32, 32, 32), (32, 32, 32, 32, 32, 32),
                    (10, 10, 10), (10, 10, 10, 10), (10, 10, 10, 10, 10)],
    DecisionTreeClassifier: range(1, 20),
    KNeighborsClassifier: range(1, 20),
    RandomForestClassifier: range(1, 20),
    GradientBoostingClassifier: range(1, 20),
    AdaBoostClassifier: range(1, 20),
    XGBClassifier: range(1, 20),
    SVC: range(1, 20),
    LogisticRegression: ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    LinearRegression: None,
    GaussianNB: None,
    MLPRegressor: [(250, 250, 250), (250, 250, 250, 250), (250, 250, 250, 250, 250), (250, 250, 250, 250, 250, 250, 250),
                    (100, 100, 100), (100, 100, 100, 100), (100, 100, 100, 100, 100), (100, 100, 100, 100, 100, 100, 100),
                    (32, 32, 32), (32, 32, 32, 32), (32, 32, 32, 32, 32), (32, 32, 32, 32, 32, 32),
                    (10, 10, 10), (10, 10, 10, 10), (10, 10, 10, 10, 10)],
    DecisionTreeRegressor: range(1, 20),
    KNeighborsRegressor: range(1, 20),
    RandomForestRegressor: range(1, 20),
    XGBRegressor: range(1, 20),
    SVR: range(1, 20)
}

other_params = {
    MLPClassifier: {'early_stopping': True, 'max_iter': 1000},
    DecisionTreeClassifier: {},
    KNeighborsClassifier: {},
    RandomForestClassifier: {},
    GradientBoostingClassifier: {},
    AdaBoostClassifier: {},
    XGBClassifier: {},
    SVC: {},
    LogisticRegression: {'max_iter': 10000},
    LinearRegression: {},
    GaussianNB: {},
    MLPRegressor: {'early_stopping': True, 'max_iter': 1000},
    DecisionTreeRegressor: {},
    KNeighborsRegressor: {},
    RandomForestRegressor: {},
    XGBRegressor: {},
    SVR: {}
}
    

model_preprocess = {
    MLPClassifier: None,
    DecisionTreeClassifier: None,
    KNeighborsClassifier: None,
    RandomForestClassifier: None,
    GradientBoostingClassifier: None,
    AdaBoostClassifier: None,
    XGBClassifier: None,
    SVC: None,
    LogisticRegression: StandardScaler(),
    LinearRegression: StandardScaler(),
    GaussianNB: None,
    MLPRegressor: StandardScaler(),
    DecisionTreeRegressor: None,
    KNeighborsRegressor: StandardScaler(),
    RandomForestRegressor: None,
    XGBRegressor: None,
    SVR: None
}

model_names = {
    MLPClassifier: 'MLP Classifier',
    DecisionTreeClassifier: 'Decision Tree Classifier',
    KNeighborsClassifier: 'K Neighbors Classifier',
    RandomForestClassifier: 'Random Forest Classifier',
    GradientBoostingClassifier: 'Gradient Boosting Classifier',
    AdaBoostClassifier: 'Ada Boost Classifier',
    XGBClassifier: 'XGBoost Classifier',
    SVC: 'SVM Classifier',
    LogisticRegression: 'Logistic Regression Classifier',
    LinearRegression: 'Linear Regression',
    GaussianNB: 'Gaussian Naive Bayes Classifier',
    MLPRegressor: 'MLP Regressor',
    DecisionTreeRegressor: 'Decision Tree Regressor',
    KNeighborsRegressor: 'K Neighbors Regressor',
    RandomForestRegressor: 'Random Forest Regressor',
    XGBRegressor: 'XGBoost Regressor',
    SVR: 'SVM Regressor'
}

for model in models:
    print("Model: ", model_names[model])
    model_accuracies[model] = {}
    model_mses[model] = {}
    for param in params[model]:
        print("Param: ", param)
        if model_preprocess[model] is not None:
            pX_train = model_preprocess[model].fit_transform(X_train)
            pX_test = model_preprocess[model].transform(X_test)
        else:
            pX_train = X_train
            pX_test = X_test
        if other_params[model] is not None:
            m = model(**other_params[model], **{'n_neighbors': param})
        else:
            m = model(param)
        m.fit(pX_train, y_train)
        y_pred = m.predict(pX_test)
        if model in [LinearRegression, GaussianNB]:
            model_mses[model][param] = mean_squared_error(y_test, y_pred)
        else:
            model_accuracies[model][param] = accuracy_score(y_test, y_pred.round())
            model_mses[model][param] = mean_squared_error(y_test, y_pred)
    print("Max Accuracy: ", max(model_accuracies[model], key=model_accuracies[model].get), model_accuracies[model][max(model_accuracies[model], key=model_accuracies[model].get)])
    print("Min MSE: ", min(model_mses[model], key=model_mses[model].get), model_mses[model][min(model_mses[model], key=model_mses[model].get)])
    print("")

Model:  MLP Classifier
Param:  (250, 250, 250)


TypeError: MLPClassifier.__init__() got an unexpected keyword argument 'n_neighbors'

In [94]:
# Print results
for key in results:
    print(key, results[key]['accuracy'], results[key]['mse'])

('MLPClassifer', (100, 100, 100)) 0.44660194174757284 1.1423948220064726
('MLPClassifer', (100, 100, 100, 100)) 0.47572815533980584 0.8478964401294499
('MLPClassifer', (100, 100, 100, 100, 100)) 0.5145631067961165 1.0550161812297734
('MLPClassifer', (100, 100, 100, 100, 100, 100)) 0.5048543689320388 0.8802588996763754
('MLPClassifer', (100, 100, 100, 100, 100, 100, 100)) 0.5307443365695793 1.0323624595469256
('MLPClassifer', (32, 32, 32)) 0.42718446601941745 1.1585760517799353
('MLPClassifer', (32, 32, 32, 32)) 0.45307443365695793 1.2944983818770226
('MLPClassifer', (32, 32, 32, 32, 32)) 0.42071197411003236 1.0906148867313916
('MLPClassifer', (32, 32, 32, 32, 32, 32)) 0.34951456310679613 1.233009708737864
('MLPClassifer', (10, 10, 10)) 0.4627831715210356 1.692556634304207
('MLPClassifer', (10, 10, 10, 10)) 0.43042071197411 1.6213592233009708
('MLPClassifer', (10, 10, 10, 10, 10)) 0.4368932038834951 1.27831715210356
('MLPClassifer', (10, 10, 10, 10, 10, 10)) 0.4368932038834951 1.0453074

In [99]:
# Highest accuracy
accuracies = []
mses = []

for key in results:
    accuracies.append((key, results[key]['accuracy']))
    mses.append((key, results[key]['mse']))
    
    
print("Max Accuracy: ", max(accuracies, key=lambda x: x[1]))
print("Min MSE: ", min(mses, key=lambda x: x[1]))


Max Accuracy:  (('XGBClassifier', 5), 0.7313915857605178)
Min MSE:  (('XGBRegressor', 2), 0.2643110258098708)
